<a href="https://colab.research.google.com/github/HindJamal97/Algorithm-for-Massive-Data/blob/main/A_priori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#FIRST PART. 
#Install kaggle and upload + unzip our dataset. 

In [2]:
!pip install -q kaggle

In [3]:
#After installing kaggle, we need to upload the kaggle.json file with the API token given from out Kaggle account. 

In [4]:
from google.colab import files
files.upload() 

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hindjamal","key":"5b2794cdd1f8c072feba270d5a765fa6"}'}

In [5]:
#Now we need to create a Kaggle folder, copy the kaggle.json to the folder created and finally we need to grant an approppriate permission for the json file to act. 

In [6]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
#To list all the available datasets we have in Kaggle we use the following command

In [8]:
! kaggle datasets list

ref                                                                   title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
arnabchaki/data-science-salaries-2023                                 Data Science Salaries 2023 💸                         25KB  2023-04-13 09:55:16          30048        828  1.0              
tawfikelmetwally/automobile-dataset                                   Car information dataset                               6KB  2023-05-28 18:26:48           2988         83  0.9411765        
fatihb/coffee-quality-data-cqi                                        Coffee Quality Data (CQI May-2023)                   22KB  2023-05-12 13:06:39           5320        110  1.0              
mohithsairamreddy/salary-data 

In [9]:
#Finally, for our aim, we need to upload our dataset, and given that the url is the following one, we can download it as follows: 

In [10]:
!kaggle datasets download -d  xhlulu/medal-emnlp

100% 6.82G/6.82G [00:58<00:00, 140MB/s]
100% 6.82G/6.82G [00:58<00:00, 125MB/s]


In [11]:
from zipfile import ZipFile


In [12]:
file_name = "medal-emnlp.zip"

with ZipFile(file_name, "r") as zip:
  zip.extractall()
  print("Done")

Done


In [13]:
#SECOND PART. 
#Install Spark and construct a software that can be used to find frequent itemset (RDD). 

In [14]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.3.2
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
# unzip it
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [15]:
!java -version

openjdk version "11.0.19" 2023-04-18
OpenJDK Runtime Environment (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [16]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.3.2-bin-hadoop3'

In [17]:
# Install library for finding Spark
!pip install -q findspark
# Import the libary
import findspark
# Initiate findspark
findspark.init()
# Check the location for Spark
findspark.find()

'/content/spark-3.3.2-bin-hadoop3'

In [18]:
!pip install pyspark
import pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=4219f66206b64f49abdf06d609faad7f2b6477eec7915281c39766f4a2c80865
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [19]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext   

In [20]:
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [21]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [22]:
sc = spark.sparkContext

In [23]:
#Upload the file

In [24]:
dataset_path = '/content/full_data.csv'

In [25]:
df  = spark.read.format('csv') \
                .option('header', True) \
                .option('multiline', True) \
                .option('quote', '"') \
                .option('escape', '"') \
                .load(dataset_path)
df.printSchema()

root
 |-- TEXT: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- LABEL: string (nullable = true)



In [26]:
df.show()

+--------------------+--------------------+--------------------+
|                TEXT|            LOCATION|               LABEL|
+--------------------+--------------------+--------------------+
|alphabisabolol ha...|                  56|           substrate|
|a report is given...|24|49|68|113|137|172|carcinosarcoma|re...|
|the virostatic co...|                  55|           substrate|
|rmi rmi and rmi a...|   25|82|127|182|222|compounds|compoun...|
|a doubleblind stu...|22|26|28|77|90|14...|oxazepam|placebo|...|
|stroma from eithe...|         6|82|84|107|red cells|serum|a...|
|the effect of the...|                4|13|major|pentose pho...|
|in one experiment...|        32|44|76|135|feeding|feeding|a...|
|the presence of a...|7|15|63|137|199|2...|active|study|acti...|
|the reaction of g...|     113|203|209|250|stable|assay|bind...|
|choline acetyltra...|                  44|             caudate|
|increasing concen...|                  81|        displacement|
|the properties of...|   

In [27]:
from pyspark.sql.functions import desc

In [28]:
df.groupBy("LABEL").count().orderBy(desc("count")).show(10)

+-----------+------+
|      LABEL| count|
+-----------+------+
|      study|294977|
|      after|114472|
|    factors| 71336|
|development| 66812|
|     cancer| 61340|
|      model| 58257|
|     levels| 50882|
|   function| 50024|
|   specific| 44586|
|   approach| 43411|
+-----------+------+
only showing top 10 rows



In [29]:
Study_Label = df.filter( (df.LABEL == "study"))

In [30]:
Study_Label.show(10)

+--------------------+--------+-----+
|                TEXT|LOCATION|LABEL|
+--------------------+--------+-----+
|the behaviour of ...|      52|study|
|fast reaction tec...|       6|study|
|the role of the d...|      75|study|
|this study uses a...|      37|study|
|a statistical T0 ...|       2|study|
|the increasing us...|      78|study|
|observation of th...|      33|study|
|this T0 investiga...|       1|study|
|in a T0 of alcoho...|       2|study|
|the authors repor...|      31|study|
+--------------------+--------+-----+
only showing top 10 rows



In [31]:
Study_Label.groupBy("TEXT").count().orderBy(desc("count")).show(10)

+--------------------+-----+
|                TEXT|count|
+--------------------+-----+
|    retrospective T0|   82|
|  a retrospective T0|   77|
|retrospective coh...|   56|
|   crosssectional T0|   50|
|prospective cohor...|   44|
|      prospective T0|   21|
|a retrospective c...|   15|
|retrospective cli...|   15|
|    a prospective T0|   14|
|      casecontrol T0|   13|
+--------------------+-----+
only showing top 10 rows



In [32]:
DF = Study_Label.select('TEXT')

In [33]:
DF.show()

+--------------------+
|                TEXT|
+--------------------+
|the behaviour of ...|
|fast reaction tec...|
|the role of the d...|
|this study uses a...|
|a statistical T0 ...|
|the increasing us...|
|observation of th...|
|this T0 investiga...|
|in a T0 of alcoho...|
|the authors repor...|
|the plantar cushi...|
|in solomon island...|
|during the wet se...|
|su is a unique ne...|
|data are presente...|
|the reconstituted...|
|the cd T0 of the ...|
|the vitamin kdepe...|
|two new methods a...|
|in this introduct...|
+--------------------+
only showing top 20 rows



In [34]:
#Pre-processing of the data that we have!!

In [35]:
!pip install sparknlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 kB 2.2 MB/s eta 0:00:00


In [36]:
import sparknlp
import math
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.ml import Pipeline
import pyspark.sql.types as T
from typing import List

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import csv
import time
from tqdm import tqdm

In [38]:
from pyspark.sql.functions import lower, col, udf

In [39]:
DF_lowercase = DF.withColumn("processed_text", lower(col("text")))

In [40]:
DF_lowercase.show()

+--------------------+--------------------+
|                TEXT|      processed_text|
+--------------------+--------------------+
|the behaviour of ...|the behaviour of ...|
|fast reaction tec...|fast reaction tec...|
|the role of the d...|the role of the d...|
|this study uses a...|this study uses a...|
|a statistical T0 ...|a statistical t0 ...|
|the increasing us...|the increasing us...|
|observation of th...|observation of th...|
|this T0 investiga...|this t0 investiga...|
|in a T0 of alcoho...|in a t0 of alcoho...|
|the authors repor...|the authors repor...|
|the plantar cushi...|the plantar cushi...|
|in solomon island...|in solomon island...|
|during the wet se...|during the wet se...|
|su is a unique ne...|su is a unique ne...|
|data are presente...|data are presente...|
|the reconstituted...|the reconstituted...|
|the cd T0 of the ...|the cd t0 of the ...|
|the vitamin kdepe...|the vitamin kdepe...|
|two new methods a...|two new methods a...|
|in this introduct...|in this in

In [41]:
from pyspark.sql.functions import regexp_replace, trim, col, lower


In [42]:
def removePunctuation(column):
   return lower(trim(regexp_replace(column,'\p{Punct}',''))).alias('sentence')

In [43]:
RemoveP = DF_lowercase.select(removePunctuation(col('processed_text')).alias("processed_text"))
RemoveP.show(5)

+--------------------+
|      processed_text|
+--------------------+
|the behaviour of ...|
|fast reaction tec...|
|the role of the d...|
|this study uses a...|
|a statistical t0 ...|
+--------------------+
only showing top 5 rows



In [44]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import trim,split,explode,col

In [45]:
tokenizer = Tokenizer(inputCol='processed_text', outputCol='words_token')
Data = RemoveP.withColumn("processed_text", trim(col("processed_text")))
DATA = tokenizer.transform(Data)
DATA.show(20)

+--------------------+--------------------+
|      processed_text|         words_token|
+--------------------+--------------------+
|the behaviour of ...|[the, behaviour, ...|
|fast reaction tec...|[fast, reaction, ...|
|the role of the d...|[the, role, of, t...|
|this study uses a...|[this, study, use...|
|a statistical t0 ...|[a, statistical, ...|
|the increasing us...|[the, increasing,...|
|observation of th...|[observation, of,...|
|this t0 investiga...|[this, t0, invest...|
|in a t0 of alcoho...|[in, a, t0, of, a...|
|the authors repor...|[the, authors, re...|
|the plantar cushi...|[the, plantar, cu...|
|in solomon island...|[in, solomon, isl...|
|during the wet se...|[during, the, wet...|
|su is a unique ne...|[su, is, a, uniqu...|
|data are presente...|[data, are, prese...|
|the reconstituted...|[the, reconstitut...|
|the cd t0 of the ...|[the, cd, t0, of,...|
|the vitamin kdepe...|[the, vitamin, kd...|
|two new methods a...|[two, new, method...|
|in this introduct...|[in, this,

In [46]:
from pyspark.ml.feature import StopWordsRemover

In [47]:
remover = StopWordsRemover(inputCol='words_token', outputCol='words_clean')
Df = remover.transform(DATA).select("processed_text",'words_clean')
Df.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [48]:
#The following code is trying to eliminate some duplicated words that we have in each sentence, but we still do not know if this is the best possile action. 

In [49]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType,StringType    

In [50]:
get_uniques=udf(lambda x: list(set(x)), ArrayType(StringType()))

DataBase = Df.withColumn("words_clean", get_uniques(Df.words_clean))
DataBase.show(10)

+--------------------+--------------------+
|      processed_text|         words_clean|
+--------------------+--------------------+
|the behaviour of ...|[behaviour, human...|
|fast reaction tec...|[protein, means, ...|
|the role of the d...|[production, appe...|
|this study uses a...|[compare, least, ...|
|a statistical t0 ...|[starting, import...|
|the increasing us...|[distribution, le...|
|observation of th...|[behaviour, certa...|
|this t0 investiga...|[appears, new, de...|
|in a t0 of alcoho...|[development, fou...|
|the authors repor...|[certain, second,...|
+--------------------+--------------------+
only showing top 10 rows



In [51]:
from pyspark.sql.functions import collect_set
from pyspark.sql.types import IntegerType

In [52]:
slen = udf(lambda s: len(s), IntegerType())
DB = DataBase.withColumn("word_count", slen(DataBase.words_clean))

In [53]:
DB.show()

+--------------------+--------------------+----------+
|      processed_text|         words_clean|word_count|
+--------------------+--------------------+----------+
|the behaviour of ...|[behaviour, human...|        50|
|fast reaction tec...|[protein, means, ...|        61|
|the role of the d...|[production, appe...|        64|
|this study uses a...|[compare, least, ...|        44|
|a statistical t0 ...|[starting, import...|        62|
|the increasing us...|[distribution, le...|        50|
|observation of th...|[behaviour, certa...|        31|
|this t0 investiga...|[appears, new, de...|        55|
|in a t0 of alcoho...|[development, fou...|         8|
|the authors repor...|[certain, second,...|        26|
|the plantar cushi...|[instance, correl...|        51|
|in solomon island...|[additional, prog...|        20|
|during the wet se...|[basis, horses, u...|        59|
|su is a unique ne...|[ill, least, unco...|        52|
|data are presente...|[radiographs, tec...|        50|
|the recon

In [54]:
import pyspark.sql.functions as F

In [55]:
DB.createOrReplaceTempView("df_view")

MeDal = spark.sql("SELECT processed_text, words_clean AS items, word_count FROM df_view")

In [56]:
MeDal.show()

+--------------------+--------------------+----------+
|      processed_text|               items|word_count|
+--------------------+--------------------+----------+
|the behaviour of ...|[behaviour, human...|        50|
|fast reaction tec...|[protein, means, ...|        61|
|the role of the d...|[production, appe...|        64|
|this study uses a...|[compare, least, ...|        44|
|a statistical t0 ...|[starting, import...|        62|
|the increasing us...|[distribution, le...|        50|
|observation of th...|[behaviour, certa...|        31|
|this t0 investiga...|[appears, new, de...|        55|
|in a t0 of alcoho...|[development, fou...|         8|
|the authors repor...|[certain, second,...|        26|
|the plantar cushi...|[instance, correl...|        51|
|in solomon island...|[additional, prog...|        20|
|during the wet se...|[basis, horses, u...|        59|
|su is a unique ne...|[ill, least, unco...|        52|
|data are presente...|[radiographs, tec...|        50|
|the recon

In [57]:
MeDal.printSchema()

root
 |-- processed_text: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- word_count: integer (nullable = true)



In [58]:
# We will convert our preprocessed dataframe into an rdd
# with the following structure (key, value)

input_rdd = MeDal.rdd.map(lambda x: (1, list(set(x[1]))))

# This is our input rdd
input_rdd.take(1)

[(1,
  ['behaviour',
   'human',
   'new',
   'applying',
   'inhibitors',
   'recovery',
   'case',
   'solely',
   'group',
   'effects',
   'linear',
   'placental',
   'acting',
   'diamine',
   'theories',
   'kidney',
   'sodium',
   'inhibition',
   'thorough',
   't0',
   'used',
   'compounds',
   'carbonyl',
   'described',
   'schiffbase',
   'several',
   'particularly',
   'reagents',
   'pig',
   'vtvo',
   'timedependent',
   'plots',
   'addition',
   'theory',
   'adequately',
   'accounted',
   'time',
   'inhibiting',
   'formation',
   'usual',
   'aminoguanidine',
   'pyruvate',
   'oxidase',
   'phenylhydrazine',
   'suggested',
   'urea',
   'log',
   'activity',
   'activesite',
   'predict'])]

In [59]:
THRESHOLD_PERCENTAGE = 0.1
n_of_baskets = input_rdd.countByKey()[1]

THRESHOLD = math.ceil(n_of_baskets * THRESHOLD_PERCENTAGE)

In [60]:
print(THRESHOLD)

29498


In [61]:
# Extract all singletons
singleton_itemsets_rdd = input_rdd \
                          .flatMap(lambda x: x[1]) \
                          .map(lambda x: (x,1))

singleton_itemsets_w_frequencies_rdd = singleton_itemsets_rdd \
                                        .reduceByKey(lambda x,y: x + y)

# Filter singletons with a frequency higher than the THRESHOLD
frequent_singleton_itemsets_rdd = singleton_itemsets_w_frequencies_rdd \
                                    .filter(lambda x: x[1] > THRESHOLD)

frequent_singleton_itemsets_rdd.take(20)

[('t0', 294977),
 ('used', 38166),
 ('two', 30838),
 ('also', 30148),
 ('study', 42329),
 ('using', 40581),
 ('patients', 58487),
 ('results', 45547),
 ('data', 29668),
 ('may', 31004),
 ('present', 38754),
 ('aim', 46949)]

In [62]:
# We can then compute frequent pairs 
# Thanks to the monotonicity property we won't generate false negatives

# Extract all frequent singleton without frequency
frequent_singleton_itemsets_wout_freq_rdd = frequent_singleton_itemsets_rdd \
                                              .map(lambda x: (1, x[0]))

# Generate all candidate pairs 
candidate_pairs_rdd = frequent_singleton_itemsets_wout_freq_rdd \
                      .join(frequent_singleton_itemsets_wout_freq_rdd) \
                      .filter(lambda x: len(set(x[1])) == len(x[1])) \
                      .map(lambda x: (tuple(sorted(x[1])), 1)) \
                      .reduceByKey(lambda x,y: x)

# Generate a list of candidate pairs 
candidate_pairs_list = candidate_pairs_rdd.map(lambda x: x[0]).collect()
broadcasted_candidate_pairs_list = sc.broadcast(candidate_pairs_list)

# Compute all possible pairs on our whole dataset
input_w_unique_key_rdd = input_rdd \
                        .map(lambda x: x[1]) \
                        .zipWithUniqueId() \
                        .flatMap(lambda x: [(x[1], word) for word in x[0]])

pair_itemsets_rdd = input_w_unique_key_rdd \
                  .join(input_w_unique_key_rdd)

# Filter only pairs that are in the candidate frequent pairs list and compute their frequency
frequent_pairs_itemsets_rdd = pair_itemsets_rdd \
                                .filter(lambda x: x[1] in broadcasted_candidate_pairs_list.value) \
                                .map(lambda x: (x[1],1)) \
                                .reduceByKey(lambda x,y: x + y) \
                                .filter(lambda x: x[1] > THRESHOLD)

frequent_pairs_itemsets_rdd.take(5)

[(('study', 't0'), 42329),
 (('also', 't0'), 30148),
 (('data', 't0'), 29668),
 (('aim', 't0'), 46949),
 (('present', 't0'), 38754)]

In [ ]:
frequent_pairs_itemsets_wout_freq_rdd = frequent_pairs_itemsets_rdd.map(lambda x: (1, x[0]))

# Generate all candidate triplets 
candidate_triplets_rdd = frequent_pairs_itemsets_wout_freq_rdd \
                      .join(frequent_pairs_itemsets_wout_freq_rdd) \
                      .map(lambda x: (1, sum(x[1], ()))) \
                      .map(lambda x: (1, tuple(set(x[1]))) ) \
                      .filter(lambda x: len(x[1]) == 3) \
                      .map(lambda x: (tuple(sorted(x[1])), 1)) \
                      .reduceByKey(lambda x,y: x)

# Generate a list of candidate triplets 
candidate_triplets_list = candidate_triplets_rdd.map(lambda x: x[0]).collect()
broadcasted_candidate_triplets_list= sc.broadcast(candidate_triplets_list)

# Compute all triplets on our whole dataset
triplets_itemsets_rdd = input_w_unique_key_rdd \
                        .map(lambda x: (x[0], (x[1], ))) \
                        .join(pair_itemsets_rdd)  \
                        .map(lambda x: (1, sum(x[1], ())))

# Filter only triplets that are in the candidate frequent triples list and compute their frequency
frequent_triplets_itemsets_rdd = triplets_itemsets_rdd \
                                .filter(lambda x: x[1] in broadcasted_candidate_triplets_list.value) \
                                .map(lambda x: (x[1],1)) \
                                .reduceByKey(lambda x,y: x + y) \
                                .filter(lambda x: x[1] > THRESHOLD)

frequent_triplets_itemsets_rdd.take(5)

[]